In [93]:
import pandas as pd
import os

In [181]:
scenario_dict = {
    'TOD': r'L:\vision2050\soundcast\integrated\final_runs\tod\tod_run_8.run_2018_10_29_15_01',
    'Stay the Course': r'L:\vision2050\soundcast\integrated\final_runs\stc\stc_run_6.run_2018_10_23_11_15',
    'RUG': r'L:\vision2050\soundcast\integrated\final_runs\rug\rug_run_5.run_2018_10_25_09_07'
}

In [194]:
# Convert the TOD to an hour
tod_lookup = {'5to6' : 5, '6to7' : 6, '7to8' : 7, '8to9' : 8, '9to10' : 9, 
              '10to14' : 12, '14to15' : 14, '15to16' : 15, '16to17' : 16,
              '17to18' : 17, '18to20' : 18, '20to5' : 20}


final_df = pd.DataFrame()
for year in ['2025','2040','2050']:
    for scen_name, scen_dir in scenario_dict.iteritems():
        print scen_name
        df = pd.read_csv(os.path.join(scen_dir,year,r'outputs\network\network_results.csv'))
        
        # Some col names vary between 2025/2040/2050
        if 'speed_limit' not in df.columns:
            df['speed_limit'] = df['data2']
        if 'auto_time' not in df.columns:
            df['auto_time'] = df['time']
        if 'length' not in df.columns:
            df['length'] = df['ij_length']
        if '@tveh' not in df.columns:
            df['@tveh'] = df['tveh']
        df['speed'] = df['length']/(df['auto_time']/60)
        df.ix[df['speed'] < 0, 'speed'] = 0
        df['speed_limit_minus_travel_speed'] = df['speed_limit']-df['speed'] 
        df.ix[df['speed_limit_minus_travel_speed'] < 0, 'speed_limit_minus_travel_speed'] = 0
        df['percent_speed_difference'] = df['speed_limit_minus_travel_speed']/df['speed_limit']
        df['Percent of Speed Limit'] = df['speed']/df['speed_limit']
        
        # convert time period to a representative hour
        tod_df = pd.DataFrame.from_dict(tod_lookup, orient='index').reset_index()
        tod_df.columns = ['tod','hour']
        df = pd.merge(df, tod_df, on='tod',how='left')
        
        # For each pair ij, find min percent of speed limit
        gb = df[['ij','Percent of Speed Limit','hour','speed_limit','speed']].groupby('ij').min()[['Percent of Speed Limit','hour','speed_limit','speed']]

        # Get hour when minimum occurs
        indices = df.groupby('ij')['Percent of Speed Limit'].idxmin
        min_hour_df = df.loc[indices][['ij','tod']]
        gb = pd.merge(gb, min_hour_df, on='ij', how='left')
                
        # Also want the daily volume for the width of the line
        df_tot = df.groupby('ij').sum()[['@tveh']].reset_index()

        gb = pd.merge(df_tot, gb, on='ij')

        # Drop rows with zero volume
        gb = gb[gb['@tveh'] > 0]

        # Split percent difference into 4 bins
        gb['Percent of Speed Limit (bin)'] = pd.cut(gb['Percent of Speed Limit'], bins=[-99999,.25,.5,.7,999999], labels=['<25%','25-50%','50-70%','>70%'])

        gb['i'] = gb['ij'].apply(lambda row: row.split('-')[0])
        gb['j'] = gb['ij'].apply(lambda row: row.split('-')[-1])
        gb['scenario'] = scen_name
        gb['year'] = year
        final_df = final_df.append(gb)

Stay the Course
RUG
TOD
Stay the Course
RUG
TOD
Stay the Course
RUG
TOD


In [195]:
final_df.to_csv(r'J:\Projects\V2050\Soundcast_Analysis\tod_max_speed_diff.csv')

In [192]:
final_df

,ij,@tveh,Percent of Speed Limit,hour,speed_limit,speed,tod,Percent of Speed Limit (bin),i,j,scenario,year
0,1-59289,432.274614,1.000000,5,15.0,15.000000,9to10,>70%,1,59289,Stay the Course,2040
1,1-59975,916.324982,1.000000,5,15.0,15.000000,9to10,>70%,1,59975,Stay the Course,2040
2,10-58316,515.694545,1.000000,5,15.0,15.000000,9to10,>70%,10,58316,Stay the Course,2040
3,10-59476,924.685186,1.000000,5,15.0,15.000000,9to10,>70%,10,59476,Stay the Course,2040
4,100-65123,332.750438,1.000000,5,15.0,15.000000,9to10,>70%,100,65123,Stay the Course,2040
5,100-65997,1669.386618,1.000000,5,15.0,15.000000,9to10,>70%,100,65997,Stay the Course,2040
6,1000-115753,41.000000,1.000000,5,15.0,15.000000,9to10,>70%,1000,115753,Stay the Course,2040
7,1000-115839,2070.875763,1.000000,5,15.0,15.000000,9to10,>70%,1000,115839,Stay the Course,2040
8,100000-100036,4395.907227,0.991145,5,30.0,29.734349,18to20,>70%,100000,100036,Stay the Course,2040
11,100000-99980,5143.432411,0.991213,5,30.0,29.736386,18to20,>70%,100000,99980,Stay the Course,2040
